In [191]:
import pandas as pd

### Slavery data PreProcessing

In [192]:
slavery_2016 = pd.read_csv("../Datasets/slavery_2016.csv")[["Country", "POPULATION", "ESTIMATED NUMBER IN MODERN SLAVERY"]]
slavery_2018 = pd.read_csv("../Datasets/slavery_2018.csv")[["Country ", "Est. prevalence of population in modern slavery (victims per 1,000 population)"]]
slavery_2023 = pd.read_csv("../Datasets/slavery_2023.csv")[["Country", "Estimated prevalence of modern slavery per 1,000 population"]]

In [193]:
slavery_2016["Estimated prevalence of modern slavery per 1,000 population"] = (slavery_2016["ESTIMATED NUMBER IN MODERN SLAVERY"]/slavery_2016["POPULATION"])*1000
slavery_2016 = slavery_2016[["Country", "Estimated prevalence of modern slavery per 1,000 population"]]

In [194]:
slavery_2016.columns = ["Country", "2016"]
slavery_2018.columns = ["Country", "2018"]
slavery_2023.columns = ["Country", "2023"]

In [195]:
# Find common entries in the common column  
# NOTE: I HAVE GIVEN MORE IMPORTANCE TO COUNTRIES IN THE SALVERY DATASET. (Since slavery data is the main comparing data here)
common_entries = set(slavery_2016['Country']).intersection(slavery_2018['Country']).intersection(slavery_2023['Country'])

In [196]:
# Filter the DataFrames to keep only rows with common entries
slavery_2016_filtered = slavery_2016[slavery_2016['Country'].isin(common_entries)]
slavery_2018_filtered = slavery_2018[slavery_2018['Country'].isin(common_entries)]
slavery_2023_filtered = slavery_2023[slavery_2023['Country'].isin(common_entries)]

In [197]:
# Merge the filtered DataFrames on the common column
temp1 = pd.merge(slavery_2016_filtered, slavery_2018_filtered, on='Country', how='inner')
slavery = pd.merge(temp1, slavery_2023_filtered, on='Country', how='inner')

In [198]:
slavery.head()

,Country,2016,2018,2023
0,Afghanistan,11.301380,22.2,13.0
1,Albania,2.959394,6.9,11.8
2,Algeria,6.259611,2.7,1.9
3,Angola,6.382384,7.2,4.1
4,Armenia,4.671968,5.3,8.9


In [199]:
slavery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Country  145 non-null    object 
 1   2016     145 non-null    float64
 2   2018     145 non-null    float64
 3   2023     139 non-null    float64
dtypes: float64(3), object(1)
memory usage: 4.7+ KB


In [200]:
slavery[slavery.isnull().any(axis=1)]

,Country,2016,2018,2023
20,Cape Verde,4.606526,4.1,NaN
44,Iceland,1.208459,2.1,NaN
64,Luxembourg,0.177936,1.5,NaN
72,Montenegro,4.019293,5.9,NaN
122,Barbados,2.112676,2.7,NaN
142,Suriname,4.604052,2.3,NaN


### Other parameters data preprocessing

In [201]:
corruption = pd.read_csv("../Datasets/corruption.csv")
democracy = pd.read_csv("../Datasets/democracy.csv")
gdppercapita = pd.read_csv("../Datasets/gdppercapita.csv")
lfpr = pd.read_csv("../Datasets/lfpr.csv")
migration = pd.read_csv("../Datasets/migration.csv")

In [202]:
lfpr = lfpr[lfpr['Type_LFPR'].isin(['Total'])]  # removing male, female entries
lfpr = lfpr[["Country", "Year", "LFPR"]]

In [203]:
lfpr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6079 entries, 2 to 18172
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Country  6079 non-null   object 
 1   Year     6079 non-null   int64  
 2   LFPR     6079 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 190.0+ KB


In [204]:
years = [2016, 2018, 2023]

In [205]:
corruption = corruption[corruption['Year'].isin(years)]
democracy = democracy[democracy['Year'].isin(years)]
gdppercapita = gdppercapita[gdppercapita['Year'].isin(years)]
lfpr = lfpr[lfpr['Year'].isin(years)]
migration = migration[migration['Year'].isin(years)]

In [206]:
# Find common entries in the common column
# common_entries = set(corruption['Country']).intersection(democracy['Country']).intersection(gdppercapita['Country']).intersection(lfpr['Country']).intersection(migration['Country'])

In [207]:
# Filter the DataFrames to keep only rows with common countries
corruption_filtered = corruption[corruption['Country'].isin(common_entries)]
democracy_filtered = democracy[democracy['Country'].isin(common_entries)]
gdppercapita_filtered = gdppercapita[gdppercapita['Country'].isin(common_entries)]
lfpr_filtered = lfpr[lfpr['Country'].isin(common_entries)]
migration_filtered = migration[migration['Country'].isin(common_entries)]

In [208]:
# Create a complete DataFrame with all combinations of Country and Year
all_combinations = pd.DataFrame([(country, year) for country in common_entries for year in years], columns=['Country', 'Year'])

In [209]:
all_combinations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Country  435 non-null    object
 1   Year     435 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 6.9+ KB


In [210]:
# Merge each DataFrame with the complete combinations to fill missing rows with NaN
corruption_complete = pd.merge(all_combinations, corruption_filtered, on=['Country', 'Year'], how='left')
democracy_complete = pd.merge(all_combinations, democracy_filtered, on=['Country', 'Year'], how='left')
gdppercapita_complete = pd.merge(all_combinations, gdppercapita_filtered, on=['Country', 'Year'], how='left')
lfpr_complete = pd.merge(all_combinations, lfpr_filtered, on=['Country', 'Year'], how='left')
migration_complete = pd.merge(all_combinations, migration_filtered, on=['Country', 'Year'], how='left')

In [211]:
# Merge the filtered DataFrames on the common column
temp2 = pd.merge(corruption_complete, democracy_complete, on=['Country', 'Year'], how='inner')
temp3 = pd.merge(temp2, gdppercapita_complete, on=['Country', 'Year'], how='inner')
temp4 = pd.merge(temp3, lfpr_complete, on=['Country', 'Year'], how='inner')
comparing_data = pd.merge(temp4, migration_complete, on=['Country', 'Year'], how='inner')

In [212]:
comparing_data.head()

,Country,Year,Corruption,Democracy score,GDP per capita,LFPR,Migration
0,Namibia,2016,52.0,6.31,10627.6350,62.596,-8596.0
1,Namibia,2018,53.0,6.25,10266.1710,60.150,-8596.0
2,Namibia,2023,49.0,6.52,NaN,NaN,-3916.0
3,Liberia,2016,37.0,5.31,1525.4580,77.982,-26822.0
4,Liberia,2018,32.0,5.35,1521.7335,77.776,-23226.0


In [213]:
comparing_data.head()

,Country,Year,Corruption,Democracy score,GDP per capita,LFPR,Migration
0,Namibia,2016,52.0,6.31,10627.6350,62.596,-8596.0
1,Namibia,2018,53.0,6.25,10266.1710,60.150,-8596.0
2,Namibia,2023,49.0,6.52,NaN,NaN,-3916.0
3,Liberia,2016,37.0,5.31,1525.4580,77.982,-26822.0
4,Liberia,2018,32.0,5.35,1521.7335,77.776,-23226.0


In [214]:
comparing_data[comparing_data.isnull().any(axis=1)]

,Country,Year,Corruption,Democracy score,GDP per capita,LFPR,Migration
2,Namibia,2023,49.0,6.52,NaN,NaN,-3916.0
5,Liberia,2023,25.0,5.57,NaN,NaN,-5000.0
8,Australia,2023,75.0,8.66,NaN,NaN,139991.0
9,Kosovo,2016,36.0,NaN,10031.048,NaN,-31383.0
10,Kosovo,2018,37.0,NaN,10755.168,NaN,-34137.0
...,...,...,...,...,...,...,...
422,Bangladesh,2023,24.0,5.87,NaN,NaN,-309977.0
425,Belarus,2023,37.0,NaN,NaN,NaN,-4282.0
428,Mozambique,2023,25.0,3.51,NaN,NaN,-5000.0
431,Luxembourg,2023,78.0,8.81,NaN,NaN,4883.0


### lfpr (male, female, both) and slavery preprocess

In [215]:
lfpr = pd.read_csv("../Datasets/lfpr.csv")

In [216]:
lfpr.head()

,Country,Year,LFPR,Type_LFPR
0,Afghanistan,1991,16.101,Female
1,Afghanistan,1991,80.990,Male
2,Afghanistan,1991,48.039,Total
3,Afghanistan,1992,16.170,Female
4,Afghanistan,1992,80.940,Male


In [217]:
lfpr_total = lfpr[lfpr['Type_LFPR'].isin(['Total'])]  # removing male, female entries
lfpr_male = lfpr[lfpr['Type_LFPR'].isin(['Male'])]
lfpr_female = lfpr[lfpr['Type_LFPR'].isin(['Female'])]

In [218]:
lfpr_total = lfpr_total[["Country", "Year", "LFPR"]]
lfpr_male = lfpr_male[["Country", "Year", "LFPR"]]
lfpr_female = lfpr_female[["Country", "Year", "LFPR"]]

In [219]:
lfpr_total = lfpr_total[lfpr_total['Year'].isin([2016, 2018])]
lfpr_male = lfpr_male[lfpr_male['Year'].isin([2016, 2018])]
lfpr_female = lfpr_female[lfpr_female['Year'].isin([2016, 2018])]

In [220]:
common_lfpr_slavery = set(slavery['Country']).intersection(lfpr_total['Country'])

In [221]:
# Filter the DataFrames to keep only rows with common entries
slavery_filtered = slavery[slavery['Country'].isin(common_lfpr_slavery)]

lfpr_total_filtered = lfpr_total[lfpr_total['Country'].isin(common_lfpr_slavery)]
lfpr_male_filtered = lfpr_male[lfpr_male['Country'].isin(common_lfpr_slavery)]
lfpr_female_filtered = lfpr_female[lfpr_female['Country'].isin(common_lfpr_slavery)]

In [222]:
# Pivot the DataFrame
lfpr_total_pivot = lfpr_total_filtered.pivot(index='Country', columns='Year', values='LFPR').reset_index()
lfpr_male_pivot = lfpr_male_filtered.pivot(index='Country', columns='Year', values='LFPR').reset_index()
lfpr_female_pivot = lfpr_female_filtered.pivot(index='Country', columns='Year', values='LFPR').reset_index()

# Rename the columns for clarity
lfpr_total_pivot.columns.name = None
lfpr_total_pivot.columns = ['Country', 2016, 2018]

lfpr_male_pivot.columns.name = None
lfpr_male_pivot.columns = ['Country', 2016, 2018]

lfpr_female_pivot.columns.name = None
lfpr_female_pivot.columns = ['Country', 2016, 2018]

In [223]:
slavery_reduced = slavery_filtered[["Country", "2016", "2018"]]
slavery_reduced.columns = ["Country", 2016, 2018]

In [224]:
slavery_reduced = slavery_reduced.sort_values(by='Country')

lfpr_total_pivot = lfpr_total_pivot.sort_values(by='Country')
lfpr_male_pivot = lfpr_male_pivot.sort_values(by='Country')
lfpr_female_pivot = lfpr_female_pivot.sort_values(by='Country')

In [225]:
slavery_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Index: 138 entries, 0 to 120
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Country  138 non-null    object 
 1   2016     138 non-null    float64
 2   2018     138 non-null    float64
dtypes: float64(2), object(1)
memory usage: 4.3+ KB


In [226]:
lfpr_total_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Country  138 non-null    object 
 1   2016     138 non-null    float64
 2   2018     138 non-null    float64
dtypes: float64(2), object(1)
memory usage: 3.4+ KB


In [227]:
lfpr_male_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Country  138 non-null    object 
 1   2016     138 non-null    float64
 2   2018     138 non-null    float64
dtypes: float64(2), object(1)
memory usage: 3.4+ KB


In [228]:
lfpr_female_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Country  138 non-null    object 
 1   2016     138 non-null    float64
 2   2018     138 non-null    float64
dtypes: float64(2), object(1)
memory usage: 3.4+ KB


In [229]:
# Standardize Data
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [230]:
value_columns = [2016, 2018]
scaler = StandardScaler()

# Z-score normalization
lfpr_total_pivot.loc[:, value_columns] = scaler.fit_transform(lfpr_total_pivot[value_columns])
lfpr_male_pivot.loc[:, value_columns] = scaler.fit_transform(lfpr_male_pivot[value_columns])
lfpr_female_pivot.loc[:, value_columns] = scaler.fit_transform(lfpr_female_pivot[value_columns])

slavery_reduced.loc[:, value_columns] = scaler.fit_transform(slavery_reduced[value_columns])

In [231]:
slavery_reduced.head()

,Country,2016,2018
0,Afghanistan,1.425598,1.786060
1,Albania,-0.479608,0.101637
2,Algeria,0.274121,-0.360754
3,Angola,0.302161,0.134665
121,Argentina,-0.232309,-0.514885


In [232]:
lfpr_total_pivot.head()

,Country,2016,2018
0,Afghanistan,-1.692985,-1.829335
1,Albania,0.005041,0.172467
2,Algeria,-2.010203,-1.972802
3,Angola,0.899857,0.811514
4,Argentina,-0.021815,0.024332


In [233]:
slavery_reduced.describe()

,2016,2018
count,1.380000e+02,1.380000e+02
mean,-1.415937e-16,5.792468e-17
std,1.003643e+00,1.003643e+00
min,-1.116280e+00,-6.249776e-01
25%,-4.992309e-01,-4.240577e-01
50%,-1.537790e-01,-2.341472e-01
75%,3.023753e-01,9.888431e-02
max,7.918369e+00,9.580649e+00


In [234]:
lfpr_total_pivot.describe()

,2016,2018
count,1.380000e+02,1.380000e+02
mean,-1.016900e-15,6.307354e-16
std,1.003643e+00,1.003643e+00
min,-3.049879e+00,-3.019356e+00
25%,-5.025409e-01,-5.712169e-01
50%,8.579451e-02,1.581477e-01
75%,7.554580e-01,7.120841e-01
max,2.055512e+00,1.895326e+00


### Finding correlation between lfpr and slavery

In [235]:
slavLfpr_total_2016_corr = slavery_reduced[2016].corr(lfpr_total_pivot[2016])
slavLfpr_male_2016_corr = slavery_reduced[2016].corr(lfpr_male_pivot[2016])
slavLfpr_female_2016_corr = slavery_reduced[2016].corr(lfpr_female_pivot[2016])

In [236]:
print("slavLfpr_total_2016_corr: ", slavLfpr_total_2016_corr)
print("slavLfpr_male_2016_corr: ", slavLfpr_male_2016_corr)
print("slavLfpr_female_2016_corr: ", slavLfpr_female_2016_corr)

slavLfpr_total_2016_corr:  0.01981696460095905
slavLfpr_male_2016_corr:  -0.022584362727162906
slavLfpr_female_2016_corr:  0.046248767279351725


In [237]:
slavLfpr_total_2018_corr = slavery_reduced[2018].corr(lfpr_total_pivot[2018])
slavLfpr_male_2018_corr = slavery_reduced[2018].corr(lfpr_male_pivot[2018])
slavLfpr_female_2018_corr = slavery_reduced[2018].corr(lfpr_female_pivot[2018])

In [238]:
print("slavLfpr_total_2016_corr: ", slavLfpr_total_2018_corr)
print("slavLfpr_male_2016_corr: ", slavLfpr_male_2018_corr)
print("slavLfpr_female_2016_corr: ", slavLfpr_female_2018_corr)

slavLfpr_total_2016_corr:  0.037305512020338306
slavLfpr_male_2016_corr:  0.005393190619142592
slavLfpr_female_2016_corr:  0.05980669007914195


### gdppercapita and slavery preprocess

In [239]:
gdppercapita = pd.read_csv("../Datasets/gdppercapita.csv")

In [240]:
gdppercapita.head()

,Country,Year,GDP per capita
0,Afghanistan,2002.0,1280.4631
1,Afghanistan,2003.0,1292.3335
2,Afghanistan,2004.0,1260.0605
3,Afghanistan,2005.0,1352.3207
4,Afghanistan,2006.0,1366.9932


In [241]:
gdppercapita = gdppercapita[gdppercapita['Year'].isin([2016, 2018])]

In [242]:
common_gdppercapita_slavery = set(slavery['Country']).intersection(gdppercapita['Country'])

In [243]:
diff_set1 = common_lfpr_slavery - common_gdppercapita_slavery
diff_set2 = common_gdppercapita_slavery - common_lfpr_slavery
print("Countries in set1 but not in set2:", diff_set1)
print("Countries in set2 but not in set1:", diff_set2)

Countries in set1 but not in set2: {'Eritrea', 'Timor-Leste', 'South Sudan', 'Cuba'}
Countries in set2 but not in set1: {'Slovakia', 'Kyrgyzstan', 'Cape Verde', 'Kosovo', 'Gambia', 'Egypt'}


In [244]:
# Filter the DataFrames to keep only rows with common entries
slavery_filtered = slavery[slavery['Country'].isin(common_gdppercapita_slavery)]

gdppercapita_filtered = gdppercapita[gdppercapita['Country'].isin(common_gdppercapita_slavery)]

In [245]:
# Pivot the DataFrame
gdppercapita_pivot = gdppercapita_filtered.pivot(index='Country', columns='Year', values='GDP per capita').reset_index()

In [246]:
slavery_reduced = slavery_filtered[["Country", "2016", "2018"]]

slavery_reduced.columns = ["Country", 2016, 2018]
gdppercapita_pivot.columns = ["Country", 2016, 2018]

slavery_reduced = slavery_reduced.sort_values(by='Country')
gdppercapita_pivot = gdppercapita_pivot.sort_values(by='Country')

In [247]:
gdppercapita_pivot.describe()

,2016,2018
count,140.000000,140.000000
mean,20634.994686,21413.408378
std,21319.948995,21759.560787
min,764.336600,740.448240
25%,4643.781700,5049.146600
50%,13043.354500,14102.327750
75%,28704.707000,31420.179500
max,116283.700000,114164.470000


In [248]:
slavery_reduced.describe()

,2016,2018
count,140.000000,140.000000
mean,5.025200,5.187143
std,4.344604,5.037555
min,0.171711,0.300000
25%,2.904559,2.175000
50%,4.542213,3.900000
75%,6.382604,6.475000
max,39.730120,40.000000


In [249]:
value_columns = [2016, 2018]
scaler = StandardScaler()

# Z-score normalization
gdppercapita_pivot.loc[:, value_columns] = scaler.fit_transform(gdppercapita_pivot[value_columns])
slavery_reduced.loc[:, value_columns] = scaler.fit_transform(slavery_reduced[value_columns])

### Finding correlation between gdppercapita and slavery

In [250]:
slavGdp_2016_corr = slavery_reduced[2016].corr(gdppercapita_pivot[2016])
slavGdp_2018_corr = slavery_reduced[2018].corr(gdppercapita_pivot[2018])

print("slavGdp_2016_corr: ", slavGdp_2016_corr)
print("slavGdp_2018_corr: ", slavGdp_2018_corr)

slavGdp_2016_corr:  0.02820868460160493
slavGdp_2018_corr:  -0.09580752609885806


### migration and slavery preprocess

In [251]:
migration = pd.read_csv("../Datasets/migration.csv")

In [252]:
migration.head()

,Country,Year,Migration
0,Afghanistan,1960,2606
1,Afghanistan,1961,6109
2,Afghanistan,1962,7016
3,Afghanistan,1963,6681
4,Afghanistan,1964,7079


In [253]:
migration = migration[migration['Year'].isin([2016, 2018, 2023])]

In [254]:
common_migration_slavery = set(slavery['Country']).intersection(migration['Country'])

In [255]:
diff_set1 = common_lfpr_slavery - common_migration_slavery
diff_set2 = common_migration_slavery - common_lfpr_slavery
print("Countries in set1 but not in set2:", diff_set1)
print("Countries in set2 but not in set1:", diff_set2)

Countries in set1 but not in set2: set()
Countries in set2 but not in set1: {'Kosovo'}


In [256]:
# Filter the DataFrames to keep only rows with common entries
slavery_filtered = slavery[slavery['Country'].isin(common_migration_slavery)]

migration_filtered = migration[migration['Country'].isin(common_migration_slavery)]

In [257]:
# Pivot the DataFrame
migration_pivot = migration_filtered.pivot(index='Country', columns='Year', values='Migration').reset_index()

In [258]:
slavery_reduced = slavery_filtered[["Country", "2016", "2018", "2023"]]

slavery_reduced.columns = ["Country", 2016, 2018, 2023]
migration_pivot.columns = ["Country", 2016, 2018, 2023]

slavery_reduced = slavery_reduced.sort_values(by='Country')
migration_pivot = migration_pivot.sort_values(by='Country')

In [259]:
migration_pivot.describe()

,2016,2018,2023
count,1.390000e+02,1.390000e+02,1.390000e+02
mean,-1.407324e+04,-9.391439e+02,-8.797396e+03
std,2.293148e+05,1.624201e+05,1.878706e+05
min,-2.290411e+06,-1.309609e+06,-9.104750e+05
25%,-2.576200e+04,-2.233050e+04,-1.500650e+04
50%,-9.820000e+02,-2.024000e+03,-4.000000e+03
75%,2.776850e+04,2.401250e+04,3.359000e+03
max,4.320170e+05,4.943640e+05,1.784718e+06


In [260]:
value_columns = [2016, 2018, 2023]
scaler = StandardScaler()

# Z-score normalization
migration_pivot.loc[:, value_columns] = scaler.fit_transform(migration_pivot[value_columns])
slavery_reduced.loc[:, value_columns] = scaler.fit_transform(slavery_reduced[value_columns])

C:\Users\msepr\AppData\Local\Temp\ipykernel_63900\1396973260.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-3.33341846e-01  1.39448509e-02 -9.69579091e-02  4.62588968e-01
  8.44166837e-02 -5.57173129e-02  1.12153255e+00  3.51681832e-01
  6.80876309e-02  1.88741431e-01 -1.79176506e-01  6.13082949e-02
  1.28370765e-01  3.10099861e-01  7.83419785e-02 -7.93115031e-02
  7.52871195e-02  4.23050865e-01 -3.82409956e-03  2.83175685e-02
 -4.17599934e-01 -1.15996070e-01  1.19586951e-01  1.46426322e+00
 -1.69307298e-01  1.55890755e-01  6.15340670e-01 -6.08261648e-01
  4.11623592e-01  7.97556166e-02  2.20683980e-03 -1.66644128e-03
  8.29417733e-02  1.61637216e-01  6.86959768e-02 -7.04970533e-02
  1.80036396e-01 -1.46290818e-01  1.36471831e-01 -1.81439203e-01
  6.66871225e-02  5.68454275e-01  1.36913866e-01  1.26055550e-01
  9.39576473e-02  1.52403212e-02  1.95235353e+00  9.22901670e-02
 -5.75992461e-02 -1.10258362e-01 -5

### Finding correlation between migration and slavery

In [261]:
slavMigration_2016_corr = slavery_reduced[2016].corr(migration_pivot[2016])
slavMigration_2018_corr = slavery_reduced[2018].corr(migration_pivot[2018])
slavMigration_2023_corr = slavery_reduced[2023].corr(migration_pivot[2023])

print("slavMigration_2016_corr: ", slavMigration_2016_corr)
print("slavMigration_2018_corr: ", slavMigration_2018_corr)
print("slavMigration_2023_corr: ", slavMigration_2023_corr)

slavMigration_2016_corr:  -0.1345895732639958
slavMigration_2018_corr:  -0.008146733551690358
slavMigration_2023_corr:  0.0040715348807804945


### corruption and slavery preprocess

In [262]:
corruption = pd.read_csv("../Datasets/corruption.csv")

In [263]:
corruption.head()

,Country,Year,Corruption
0,Afghanistan,2012,8.0
1,Afghanistan,2013,8.0
2,Afghanistan,2014,12.0
3,Afghanistan,2015,11.0
4,Afghanistan,2016,15.0


In [264]:
corruption = corruption[corruption['Year'].isin([2016, 2018, 2023])]

In [265]:
common_corruption_slavery = set(slavery['Country']).intersection(corruption['Country'])

In [266]:
diff_set1 = common_lfpr_slavery - common_corruption_slavery
diff_set2 = common_corruption_slavery - common_lfpr_slavery
print("Countries in set1 but not in set2:", diff_set1)
print("Countries in set2 but not in set1:", diff_set2)

Countries in set1 but not in set2: set()
Countries in set2 but not in set1: {'Yemen', 'Kyrgyzstan', 'Kosovo', 'Gambia', 'Egypt'}


In [267]:
# Equatorial Guinea has Nan values
common_corruption_slavery.remove("Equatorial Guinea")

In [268]:
# Filter the DataFrames to keep only rows with common entries
slavery_filtered = slavery[slavery['Country'].isin(common_corruption_slavery)]

corruption_filtered = corruption[corruption['Country'].isin(common_corruption_slavery)]

In [269]:
# Pivot the DataFrame
corruption_pivot = corruption_filtered.pivot(index='Country', columns='Year', values='Corruption').reset_index()

In [270]:
slavery_reduced = slavery_filtered[["Country", "2016", "2018", "2023"]]

slavery_reduced.columns = ["Country", 2016, 2018, 2023]
corruption_pivot.columns = ["Country", 2016, 2018, 2023]

slavery_reduced = slavery_reduced.sort_values(by='Country')
corruption_pivot = corruption_pivot.sort_values(by='Country')

In [271]:
# Providing new rankings

# Replace the values with ranks (1 for the lowest, etc.)
slavery_ranked = slavery_reduced.copy()
slavery_ranked.iloc[:, 1:] = slavery_reduced.iloc[:, 1:].rank(axis=0, method='min')

corruption_ranked = corruption_pivot.copy()
corruption_ranked.iloc[:, 1:] = corruption_pivot.iloc[:, 1:].rank(axis=0, method='min')

In [272]:
corruption_ranked.describe()

,2016,2018,2023
count,142.000000,142.000000,142.000000
mean,70.373239,70.366197,70.352113
std,41.520500,41.358231,41.466076
min,1.000000,1.000000,1.000000
25%,36.000000,35.000000,36.000000
50%,70.000000,69.500000,71.500000
75%,106.500000,106.000000,105.000000
max,141.000000,142.000000,142.000000


In [273]:
slavery_ranked.describe()

,2016,2018,2023
count,142.000000,142.000000,137.000000
mean,71.500000,70.704225,68.357664
std,41.135953,41.426135,39.805360
min,1.000000,1.000000,1.000000
25%,36.250000,36.000000,35.000000
50%,71.500000,71.000000,69.000000
75%,106.750000,106.500000,103.000000
max,142.000000,142.000000,137.000000


### Finding correlation between migration and slavery

In [274]:
slavCorruption_2016_corr = slavery_ranked[2016].corr(corruption_ranked[2016])
slavCorruption_2018_corr = slavery_ranked[2018].corr(corruption_ranked[2018])
slavCorruption_2023_corr = slavery_ranked[2023].corr(corruption_ranked[2023])

print("slavCorruption_2016_corr: ", slavCorruption_2016_corr)
print("slavCorruption_2018_corr: ", slavCorruption_2018_corr)
print("slavCorruption_2023_corr: ", slavCorruption_2023_corr)

slavCorruption_2016_corr:  -0.17703288868372158
slavCorruption_2018_corr:  -0.026178850810978035
slavCorruption_2023_corr:  -0.08792134500974737


### Democracy and slavery preprocess

In [275]:
democracy = pd.read_csv("../Datasets/democracy.csv")

In [276]:
democracy.head()

,Country,Year,Democracy score
0,Afghanistan,2006,3.06
1,Afghanistan,2008,3.02
2,Afghanistan,2010,2.48
3,Afghanistan,2011,2.48
4,Afghanistan,2012,2.48


In [277]:
democracy = democracy[democracy['Year'].isin([2016, 2018, 2023])]

In [278]:
common_democracy_slavery = set(slavery['Country']).intersection(democracy['Country'])

In [279]:
diff_set1 = common_lfpr_slavery - common_democracy_slavery
diff_set2 = common_democracy_slavery - common_lfpr_slavery
print("Countries in set1 but not in set2:", diff_set1)
print("Countries in set2 but not in set1:", diff_set2)

Countries in set1 but not in set2: {'Timor-Leste', 'Somalia', 'Barbados', 'South Sudan'}
Countries in set2 but not in set1: {'Yemen', 'Slovakia', 'Kyrgyzstan', 'Cape Verde', 'Gambia', 'Egypt'}


In [280]:
# Filter the DataFrames to keep only rows with common entries
slavery_filtered = slavery[slavery['Country'].isin(common_democracy_slavery)]

democracy_filtered = democracy[democracy['Country'].isin(common_democracy_slavery)]

In [281]:
# Pivot the DataFrame
democracy_pivot = democracy_filtered.pivot(index='Country', columns='Year', values='Democracy score').reset_index()

democracy_final = democracy_pivot.dropna()

In [282]:
slavery_reduced = slavery_filtered[["Country", "2016", "2018", "2023"]]

slavery_reduced.columns = ["Country", 2016, 2018, 2023]
democracy_final.columns = ["Country", 2016, 2018, 2023]

slavery_reduced = slavery_reduced.sort_values(by='Country')
democracy_final = democracy_final.sort_values(by='Country')

In [283]:
democracy_final.describe()

,2016,2018,2023
count,126.000000,126.000000,126.000000
mean,6.030873,5.998571,5.776984
std,1.954127,1.951131,2.151253
min,1.980000,1.980000,2.120000
25%,4.372500,4.487500,3.670000
50%,6.345000,6.245000,6.115000
75%,7.500000,7.470000,7.380000
max,9.930000,9.870000,9.810000


In [284]:
value_columns = [2016, 2018, 2023]
scaler = StandardScaler()

# Z-score normalization
democracy_final.loc[:, value_columns] = scaler.fit_transform(democracy_final[value_columns])
slavery_reduced.loc[:, value_columns] = scaler.fit_transform(slavery_reduced[value_columns])

### Finding correlation between democracy and slavery

In [285]:
slavDemocracy_2016_corr = slavery_reduced[2016].corr(democracy_final[2016])
slavDemocracy_2018_corr = slavery_reduced[2018].corr(democracy_final[2018])
slavDemocracy_2023_corr = slavery_reduced[2023].corr(democracy_final[2023])

print("slavDemocracy_2016_corr: ", slavDemocracy_2016_corr)
print("slavDemocracy_2018_corr: ", slavDemocracy_2018_corr)
print("slavDemocracy_2023_corr: ", slavDemocracy_2023_corr)

slavDemocracy_2016_corr:  -0.07872211202444274
slavDemocracy_2018_corr:  0.018214079821583018
slavDemocracy_2023_corr:  0.004806334640026755
